# Homework 3

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv -O housing.csv

--2022-09-23 08:17:03--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  6.37MB/s    in 0.2s    

2022-09-23 08:17:04 (6.37 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [2]:
import pandas as pd
import numpy as np

## Data preparation

In [3]:
df = pd.read_csv('housing.csv')

In [4]:
# Columns selection
columns = [
    'latitude',
    'longitude',
    'housing_median_age',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_income',
    'median_house_value',
    'ocean_proximity'
]
df = df[columns]

In [5]:
# Missing values
df = df.fillna(0)

# Rooms per household
df['rooms_per_household'] = df['total_rooms'] / df['households']

# Bedrooms per room
df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']

# Population per houehold
df['population_per_household'] = df['population'] / df['households']

In [6]:
df.head()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842
2,37.85,-122.24,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260
3,37.85,-122.25,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945
4,37.85,-122.25,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467


## Question 1

What is the most frequent observation (mode) for the column `ocean_proximity`?

In [7]:
df['ocean_proximity'].value_counts().sort_values(ascending=False)

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

### Split the data

- Split your data in train/val/test sets, with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
- Make sure that the target value (`median_house_value`) is not in your dataframe.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

(12384, 4128, 4128)

In [10]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.median_house_value.values
y_val = df_val.median_house_value.values
y_test = df_test.median_house_value.values

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

## Question 2

- Create the correlation matrix for the numerical features of your train dataset.
- In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
- What are the two features that have the biggest correlation in this dataset?

In [11]:
numerical = [
              'latitude', 
              'longitude', 
              'housing_median_age', 
              'total_rooms',
              'total_bedrooms', 
              'population', 
              'households', 
              'median_income',
              'rooms_per_household',
              'bedrooms_per_room', 
              'population_per_household'
       ]
categorical = ['ocean_proximity']

In [12]:
df_train.corr()

/tmp/ipykernel_22077/299540020.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_train.corr()


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
latitude,1.000000,-0.925005,0.002477,-0.025914,-0.059730,-0.100272,-0.063529,-0.076805,0.119118,-0.124507,-0.002301
longitude,-0.925005,1.000000,-0.099812,0.036449,0.063840,0.091670,0.049762,-0.016426,-0.034814,0.102320,0.011022
housing_median_age,0.002477,-0.099812,1.000000,-0.363522,-0.324156,-0.292476,-0.306119,-0.119591,-0.181275,0.129456,0.012167
total_rooms,-0.025914,0.036449,-0.363522,1.000000,0.931546,0.853219,0.921441,0.198951,0.168926,-0.194185,-0.029452
total_bedrooms,-0.059730,0.063840,-0.324156,0.931546,1.000000,0.877340,0.979399,-0.009833,0.010381,0.078094,-0.034301
population,-0.100272,0.091670,-0.292476,0.853219,0.877340,1.000000,0.906841,-0.000849,-0.076210,0.031592,0.064998
households,-0.063529,0.049762,-0.306119,0.921441,0.979399,0.906841,1.000000,0.011925,-0.085832,0.058004,-0.032522
median_income,-0.076805,-0.016426,-0.119591,0.198951,-0.009833,-0.000849,0.011925,1.000000,0.394154,-0.616617,-0.000454
rooms_per_household,0.119118,-0.034814,-0.181275,0.168926,0.010381,-0.076210,-0.085832,0.394154,1.000000,-0.500589,0.001801
bedrooms_per_room,-0.124507,0.102320,0.129456,-0.194185,0.078094,0.031592,0.058004,-0.616617,-0.500589,1.000000,-0.002851


Largest correlation: `total_bedrooms` and `households`

## Make `median_house_value` binary

- We need to turn the `median_house_value` variable from numeric into binary.
- Let's create a variable `above_average` which is 1 if the `median_house_value` is above its mean value and 0 otherwise.

In [13]:
average = y_train.mean()
y_above_average_train = (y_train > average).astype(int)
y_above_average_val = (y_val > average).astype(int)
y_above_average_test = (y_test > average).astype(int)


## Question 3

- Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
- What is the value of mutual information?
- Round it to 2 decimal digits using round(score, 2)

In [14]:
from sklearn.metrics import mutual_info_score

In [15]:
round(mutual_info_score(df_train[categorical[0]], y_above_average_train), 2)

0.1

## Question 4

- Now let's train a logistic regression
- Remember that we have one categorical variable `ocean_proximity` in the data. Include it using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)`
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [16]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [17]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [18]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_above_average_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [19]:
y_pred = model.predict(X_val)
accuracy = (y_pred == y_above_average_val).mean()
round(accuracy, 2)

0.84

## Question 5

- Let's find the least useful feature using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?
    - `total_rooms`
    - `total_bedrooms`
    - `population`
    - `households`

In [20]:
accuracy_differences = []
all_features = categorical + numerical

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)

for f in all_features:
    features = all_features.copy()
    features.remove(f)

    train_dict = df_train[features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[features].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    model.fit(X_train, y_above_average_train)

    y_pred = model.predict(X_val)
    accuracy_differences.append(abs(accuracy - (y_pred == y_above_average_val).mean()))

print(sorted(list(zip(all_features, accuracy_differences)), key=lambda k: k[1]))

[('total_bedrooms', 0.0004844961240310086), ('population_per_household', 0.0004844961240310086), ('total_rooms', 0.0007267441860464574), ('rooms_per_household', 0.0007267441860465684), ('bedrooms_per_room', 0.0014534883720930258), ('latitude', 0.002422480620155043), ('households', 0.0031492248062016115), ('housing_median_age', 0.004844961240310086), ('longitude', 0.005087209302325646), ('population', 0.009932170542635732), ('ocean_proximity', 0.016715116279069853), ('median_income', 0.04990310077519389)]


## Question 6

- For this question, we'll see how to use a linear regression model from Scikit-Learn
- We'll need to use the original column `median_house_value`. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model (`model = Ridge(alpha=a, solver="sag", random_state=42)`) on the training data.
- This model has a parameter alpha. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
- Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [21]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [22]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

alpha = [0, 0.01, 0.1, 1, 10]
rmse = []

y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

for a in alpha:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    model.fit(X_train, y_train_log)
    
    y_pred = model.predict(X_val)
    rmse.append(round(mean_squared_error(y_val_log, y_pred, squared=False), 3))

print(sorted(list(zip(alpha, rmse)), key=lambda k: k[1], reverse=True))

[(0, 0.524), (0.01, 0.524), (0.1, 0.524), (1, 0.524), (10, 0.524)]
